In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import networkx as nx
from scipy.spatial.distance import pdist, squareform

communication_radius = 1.5

def get_communication_network(positions, radius):  # Fixed spelling
    distances = squareform(pdist(positions))
    adjacency_matrix = (distances <= radius).astype(int)
    np.fill_diagonal(adjacency_matrix, 0)  # Remove self-connections
    return adjacency_matrix

from matplotlib.collections import LineCollection

# Add this global variable at the top
network_collection = None

def visualize_network(positions, adjacency_matrix, ax):
    global network_collection
    
    # Remove previous network collection
    if network_collection is not None:
        network_collection.remove()
    
    # Collect all line segments
    segments = []
    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            if adjacency_matrix[i, j] == 1:
                segments.append([positions[i], positions[j]])
    
    # Draw all lines at once
    if segments:
        network_collection = LineCollection(segments, colors='gray', alpha=0.3, linewidths=0.5)
        ax.add_collection(network_collection)
    else:
        network_collection = None
num_agents = 50
L = 10  # Size of the area (LxL)
v = 0.1  # Speed of the agents
eta = 0.1  # Noise parameter
num_steps = 200

# Initialize agent positions and directions randomly
positions = np.random.rand(num_agents, 2) * L
directions = np.random.rand(num_agents) * 2 * np.pi

def update_positions():
    global positions, directions
    new_positions = np.zeros_like(positions)
    new_directions = np.zeros_like(directions)

    for i in range(num_agents):
        # Add noise
        new_directions[i] = directions[i] + eta * (np.random.rand() - 0.5)

        # Update positions based on the new direction
        new_positions[i] = positions[i] + v * np.array([np.cos(new_directions[i]), np.sin(new_directions[i])])
        new_positions[i] %= L  # Ensure agents stay within bounds (periodic boundary)

    positions[:] = new_positions
    directions[:] = new_directions

# Set up the plot
fig, ax = plt.subplots()
scat = ax.scatter(positions[:, 0], positions[:, 1], c='blue')
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_aspect('equal')

def animate(frame):
    update_positions()
    
    # Update particle positions
    scat.set_offsets(positions)
    
    # Add network visualization
    adjacency_matrix = get_communication_network(positions, communication_radius)
    visualize_network(positions, adjacency_matrix, ax)
    
    return scat,

# Create the animation
ani = FuncAnimation(fig, animate, frames=num_steps, interval=50, blit=True)

# Display the animation in Jupyter Notebook
from IPython.display import HTML

# Use JavaScript animation to avoid ffmpeg issues
HTML(ani.to_jshtml())

The Visualization I have above is showing gray lines between particles that are less than 1.5 units away from one another, rather than the 6 nearest neighbors. You'll see a visualization of each particles' 6 nearest neighbors in the visualization below. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.spatial.distance import pdist, squareform

# Parameters
L = 10  # Size of the area (LxL)
v = 0.1  # Speed of the agents
eta = 0.3  # Noise parameter (reduced for better alignment)
num_steps = 100
alignment_strength = 0.7  # How strongly to align with neighbors (0-1)
num_agents = 100

# Initialize agent positions and directions randomly
positions = np.random.rand(num_agents, 2) * L
directions = np.random.rand(num_agents) * 2 * np.pi

# Global variable to track network lines
network_lines = []

def get_nearest_neighbors(positions, k=6):
    """Get the k nearest neighbors for each agent"""
    distances = squareform(pdist(positions))
    np.fill_diagonal(distances, np.inf)  # Exclude self
    nearest_indices = np.argsort(distances, axis=1)[:, :k]
    return nearest_indices

def visualize_nearest_neighbor_network(positions, nearest_neighbors, ax):
    """Draw gray lines between each particle and its 6 nearest neighbors"""
    global network_lines
    
    # Remove previous network lines
    for line in network_lines:
        line.remove()
    network_lines.clear()
    
    # Draw new network edges
    for i in range(len(positions)):
        for neighbor in nearest_neighbors[i]:
            line = ax.plot([positions[i, 0], positions[neighbor, 0]],
                          [positions[i, 1], positions[neighbor, 1]],
                          'gray', alpha=0.4, linewidth=0.8)[0]
            network_lines.append(line)

def update_positions_with_alignment():
    global positions, directions
    new_positions = np.zeros_like(positions)
    new_directions = np.zeros_like(directions)
    
    # Get nearest neighbors for each agent
    nearest_neighbors = get_nearest_neighbors(positions, k=6)
    
    for i in range(num_agents):
        # Get the 6 nearest neighbors
        neighbors = nearest_neighbors[i]
        
        # Calculate average direction of neighbors
        neighbor_directions = directions[neighbors]
        
        # Convert to unit vectors
        neighbor_vectors = np.column_stack([
            np.cos(neighbor_directions),
            np.sin(neighbor_directions)
        ])
        
        # Average direction vector
        avg_direction_vector = np.mean(neighbor_vectors, axis=0)
        
        # Convert back to angle
        avg_direction = np.arctan2(avg_direction_vector[1], avg_direction_vector[0])
        
        # Align with neighbors
        new_directions[i] = (1 - alignment_strength) * directions[i] + alignment_strength * avg_direction
        
        # Add noise
        new_directions[i] += eta * (np.random.rand() - 0.5)
        
        # Update position
        new_positions[i] = positions[i] + v * np.array([
            np.cos(new_directions[i]), 
            np.sin(new_directions[i])
        ])
        new_positions[i] %= L  # Periodic boundary
    
    positions[:] = new_positions
    directions[:] = new_directions

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 10))
scat = ax.scatter(positions[:, 0], positions[:, 1], c='blue', s=50)
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_aspect('equal')
ax.set_title('Particle Alignment Model - 6 Nearest Neighbors with Network Visualization')
order_parameter_list = []
def animate(frame):
    update_positions_with_alignment()
    
    # Update particle positions
    scat.set_offsets(positions)
    
    # Update network visualization
    nearest_neighbors = get_nearest_neighbors(positions, k=6)
    visualize_nearest_neighbor_network(positions, nearest_neighbors, ax)
    
    # Calculate and display order parameter
    current_order = calculate_polar_order_parameter(directions)
    order_parameter_list.append(current_order)
    ax.set_title(f'Frame {frame}: Order Parameter = {current_order:.3f}')
    
    return scat,

def calculate_polar_order_parameter(directions):
    """Calculate the polar order parameter (0 = random, 1 = perfectly aligned)"""
    # Convert directions to unit vectors
    vectors = np.column_stack([np.cos(directions), np.sin(directions)])
    
    # Calculate average direction vector
    avg_vector = np.mean(vectors, axis=0)
    
    # Magnitude of average vector = order parameter
    order_parameter = np.linalg.norm(avg_vector)
    
    return order_parameter

# Create the animation
ani = FuncAnimation(fig, animate, frames=num_steps, interval=100, blit=True)

# Display the animation
from IPython.display import HTML
HTML(ani.to_jshtml())




The order parameter is based on the mean of each particle's unit vector representing the direction its moving in. As the mean approaches 1, the particles are generally moving more and more in the same direction. 

In [ ]:

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(order_parameter_list, linewidth=2, color='blue', marker='o', markersize=4)

# Customize the plot
plt.xlabel('Frame Number', fontsize=12)
plt.ylabel('Order Parameter', fontsize=12)
plt.title('Evolution of Order Parameter Over Time', fontsize=14)
plt.grid(True, alpha=0.3)

# Set axis limits
plt.xlim(0, len(order_parameter_list) - 1)
plt.ylim(0, 1.0)

# Add some styling
plt.tight_layout()
plt.show()

# Optional: Print some statistics
print(f"Initial Order Parameter: {order_parameter_list[0]:.3f}")
print(f"Final Order Parameter: {order_parameter_list[-1]:.3f}")
print(f"Maximum Order Parameter: {max(order_parameter_list):.3f}")
print(f"Average Order Parameter: {np.mean(order_parameter_list):.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

def get_nearest_neighbors(positions, k=6):
    """Get the k nearest neighbors for each agent"""
    distances = squareform(pdist(positions))
    np.fill_diagonal(distances, np.inf)  # Exclude self
    nearest_indices = np.argsort(distances, axis=1)[:, :k]
    return nearest_indices

def calculate_polar_order_parameter(directions):
    """Calculate the polar order parameter (0 = random, 1 = perfectly aligned)"""
    # Convert directions to unit vectors
    vectors = np.column_stack([np.cos(directions), np.sin(directions)])
    
    # Calculate average direction vector
    avg_vector = np.mean(vectors, axis=0)
    
    # Magnitude of average vector = order parameter
    order_parameter = np.linalg.norm(avg_vector)
    
    return order_parameter

def run_simulation(eta, alignment_strength, num_steps=100, num_agents=100, L=10, v=0.1):
    """Run a single simulation with given parameters"""
    # Initialize agent positions and directions randomly
    positions = np.random.rand(num_agents, 2) * L
    directions = np.random.rand(num_agents) * 2 * np.pi
    
    order_parameter_list = []
    
    for step in range(num_steps):
        new_positions = np.zeros_like(positions)
        new_directions = np.zeros_like(directions)
        
        # Get nearest neighbors for each agent
        nearest_neighbors = get_nearest_neighbors(positions, k=6)
        
        for i in range(num_agents):
            # Get the 6 nearest neighbors
            neighbors = nearest_neighbors[i]
            
            # Calculate average direction of neighbors
            neighbor_directions = directions[neighbors]
            
            # Convert to unit vectors
            neighbor_vectors = np.column_stack([
                np.cos(neighbor_directions),
                np.sin(neighbor_directions)
            ])
            
            # Average direction vector
            avg_direction_vector = np.mean(neighbor_vectors, axis=0)
            
            # Convert back to angle
            avg_direction = np.arctan2(avg_direction_vector[1], avg_direction_vector[0])
            
            # Align with neighbors
            new_directions[i] = (1 - alignment_strength) * directions[i] + alignment_strength * avg_direction
            
            # Add noise
            new_directions[i] += eta * (np.random.rand() - 0.5)
            
            # Update position
            new_positions[i] = positions[i] + v * np.array([
                np.cos(new_directions[i]), 
                np.sin(new_directions[i])
            ])
            new_positions[i] %= L  # Periodic boundary
        
        positions[:] = new_positions
        directions[:] = new_directions
        
        # Calculate and store order parameter
        current_order = calculate_polar_order_parameter(directions)
        order_parameter_list.append(current_order)
    
    return order_parameter_list

# Define 5 different parameter combinations
trials = [
    {"eta": 0.05, "alignment_strength": 0.7, "label": "Low Noise, High Alignment"},
    {"eta": 0.15, "alignment_strength": 0.7, "label": "Medium Noise, High Alignment"},
    {"eta": 0.05, "alignment_strength": 0.4, "label": "Low Noise, Medium Alignment"},
    {"eta": 0.15, "alignment_strength": 0.4, "label": "Medium Noise, Medium Alignment"},
    {"eta": 0.25, "alignment_strength": 0.3, "label": "High Noise, Low Alignment"}
]

# Run all trials and store results
all_results = {}
num_steps = 150  # Increased for better visualization

print("Running simulations...")
for i, trial in enumerate(trials):
    print(f"Trial {i+1}/5: η={trial['eta']}, alignment={trial['alignment_strength']}")
    order_params = run_simulation(
        eta=trial['eta'],
        alignment_strength=trial['alignment_strength'],
        num_steps=num_steps
    )
    all_results[trial['label']] = order_params

# Create the comparison plot
plt.figure(figsize=(14, 8))

colors = ['blue', 'red', 'green', 'orange', 'purple']
for i, (label, order_params) in enumerate(all_results.items()):
    plt.plot(order_params, 
             linewidth=2, 
             color=colors[i], 
             label=f"{label} (η={trials[i]['eta']}, α={trials[i]['alignment_strength']})",
             alpha=0.8)

# Customize the plot
plt.xlabel('Time Steps', fontsize=12)
plt.ylabel('Order Parameter', fontsize=12)
plt.title('Order Parameter Evolution: Effect of Noise and Alignment Strength', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(0, num_steps - 1)
plt.ylim(0, 1.0)

# Add some styling
plt.tight_layout()
plt.show()

# Print summary statistics for each trial
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

for label, order_params in all_results.items():
    print(f"\n{label}:")
    print(f"  Initial Order Parameter: {order_params[0]:.3f}")
    print(f"  Final Order Parameter: {order_params[-1]:.3f}")
    print(f"  Maximum Order Parameter: {max(order_params):.3f}")
    print(f"  Average Order Parameter: {np.mean(order_params):.3f}")
    print(f"  Standard Deviation: {np.std(order_params):.3f}")
    
    # Calculate smoothness (lower std dev = smoother)
    smoothness = 1 / (1 + np.std(order_params))
    print(f"  Smoothness Index: {smoothness:.3f} (higher = smoother)")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.spatial.distance import pdist, squareform
import random

# Simulation parameters
num_agents = 100
L = 10  # Size of the area (LxL)
v = 0.1  # Speed of the agents
eta = 0.15  # Noise parameter
alignment_strength = 0.7  # Alignment strength
num_steps = 300

# Infection parameters
infection_chance = 0.3
infection_duration = 10
initial_infections = 5
neighbor_infection_threshold = 6  # Use same as flocking neighbors

class FlockingInfectionSimulation:
    def __init__(self):
        # Initialize positions and directions
        self.positions = np.random.rand(num_agents, 2) * L
        self.directions = np.random.rand(num_agents) * 2 * np.pi
        
        # Initialize infection status
        self.health_status = {
            "infected": set(random.sample(list(range(num_agents)), initial_infections)),
            "recovered": set()
        }
        self.infection_status = {}
        for i in self.health_status["infected"]:
            self.infection_status[i] = 0
            
        # Data collection
        self.susceptible_count = []
        self.infected_count = []
        self.recovered_count = []
        
        # Visualization elements
        self.network_lines = []
    
    def get_nearest_neighbors(self, k=6):
        """Get the k nearest neighbors for each agent"""
        distances = squareform(pdist(self.positions))
        np.fill_diagonal(distances, np.inf)
        nearest_indices = np.argsort(distances, axis=1)[:, :k]
        return nearest_indices
    
    def update_flocking(self):
        """Update positions and directions based on flocking rules"""
        new_positions = np.zeros_like(self.positions)
        new_directions = np.zeros_like(self.directions)
        
        nearest_neighbors = self.get_nearest_neighbors(k=6)
        
        for i in range(num_agents):
            neighbors = nearest_neighbors[i]
            neighbor_directions = self.directions[neighbors]
            
            # Calculate average direction of neighbors
            neighbor_vectors = np.column_stack([
                np.cos(neighbor_directions),
                np.sin(neighbor_directions)
            ])
            avg_direction_vector = np.mean(neighbor_vectors, axis=0)
            avg_direction = np.arctan2(avg_direction_vector[1], avg_direction_vector[0])
            
            # Align with neighbors
            new_directions[i] = ((1 - alignment_strength) * self.directions[i] + 
                               alignment_strength * avg_direction)
            
            # Add noise
            new_directions[i] += eta * (np.random.rand() - 0.5)
            
            # Update position
            new_positions[i] = self.positions[i] + v * np.array([
                np.cos(new_directions[i]), 
                np.sin(new_directions[i])
            ])
            new_positions[i] %= L  # Periodic boundary
        
        self.positions[:] = new_positions
        self.directions[:] = new_directions
    
    def update_infection(self):
        """Update infection status"""
        nearest_neighbors = self.get_nearest_neighbors(k=neighbor_infection_threshold)
        new_infections = []
        newly_recovered = []
        
        # Process current infections
        for i in list(self.health_status["infected"]):
            if self.infection_status[i] >= infection_duration:
                newly_recovered.append(i)
                continue
            
            # Try to infect neighbors
            for j in nearest_neighbors[i]:
                if (j not in self.health_status["infected"] and 
                    j not in self.health_status["recovered"]):
                    if random.random() < infection_chance:
                        new_infections.append(j)
            
            self.infection_status[i] += 1
        
        # Process recoveries
        for agent in newly_recovered:
            self.health_status["infected"].remove(agent)
            self.health_status["recovered"].add(agent)
            del self.infection_status[agent]
        
        # Process new infections
        self.health_status["infected"].update(new_infections)
        for agent in new_infections:
            self.infection_status[agent] = 0
    
    def get_colors(self):
        """Get colors for each agent based on health status"""
        colors = []
        for i in range(num_agents):
            if i in self.health_status["infected"]:
                colors.append('red')
            elif i in self.health_status["recovered"]:
                colors.append('green')  # Note: using green for recovered as is standard
            else:
                colors.append('blue')  # susceptible
        return colors
    
    def collect_data(self):
        """Collect population counts"""
        susceptible = num_agents - len(self.health_status["infected"]) - len(self.health_status["recovered"])
        infected = len(self.health_status["infected"])
        recovered = len(self.health_status["recovered"])
        
        self.susceptible_count.append(susceptible)
        self.infected_count.append(infected)
        self.recovered_count.append(recovered)
    
    def visualize_network(self, ax):
        """Draw network connections"""
        # Remove previous network lines
        for line in self.network_lines:
            line.remove()
        self.network_lines.clear()
        
        nearest_neighbors = self.get_nearest_neighbors(k=6)
        
        # Draw new network edges
        for i in range(num_agents):
            for neighbor in nearest_neighbors[i]:
                line = ax.plot([self.positions[i, 0], self.positions[neighbor, 0]],
                              [self.positions[i, 1], self.positions[neighbor, 1]],
                              'gray', alpha=0.2, linewidth=0.5)[0]
                self.network_lines.append(line)

# Create simulation instance
sim = FlockingInfectionSimulation()

# Set up the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Main simulation plot
scat = ax1.scatter(sim.positions[:, 0], sim.positions[:, 1], 
                   c=sim.get_colors(), s=50, alpha=0.8)
ax1.set_xlim(0, L)
ax1.set_ylim(0, L)
ax1.set_aspect('equal')
ax1.set_title('Flocking + Infection Simulation')
ax1.legend(['Susceptible (Blue)', 'Infected (Red)', 'Recovered (Green)'])

# Population plot
line_s, = ax2.plot([], [], 'b-', label='Susceptible', linewidth=2)
line_i, = ax2.plot([], [], 'r-', label='Infected', linewidth=2)
line_r, = ax2.plot([], [], 'g-', label='Recovered', linewidth=2)
ax2.set_xlim(0, num_steps)
ax2.set_ylim(0, num_agents)
ax2.set_xlabel('Time Steps')
ax2.set_ylabel('Number of Agents')
ax2.set_title('Population Dynamics')
ax2.legend()
ax2.grid(True, alpha=0.3)

def animate(frame):
    # Collect data for frame 0 (initial state) before any updates
    if frame == 0:
        sim.collect_data()
        # Update plots with initial state
        scat.set_offsets(sim.positions)
        scat.set_color(sim.get_colors())
        sim.visualize_network(ax1)
        
        # Set initial population plot data
        line_s.set_data([0], sim.susceptible_count)
        line_i.set_data([0], sim.infected_count) 
        line_r.set_data([0], sim.recovered_count)
        
        # Update title with initial counts
        ax1.set_title(f'Step {frame}: S={sim.susceptible_count[0]}, I={sim.infected_count[0]}, R={sim.recovered_count[0]}')
        return scat,
    
    # For all other frames, update first, then display
    sim.update_flocking()
    sim.update_infection()
    sim.collect_data()
    
    # Update main plot
    scat.set_offsets(sim.positions)
    scat.set_color(sim.get_colors())
    
    # Update network visualization
    sim.visualize_network(ax1)
    

    # Update population plot
    line_s.set_data(range(len(sim.susceptible_count)), sim.susceptible_count)
    line_i.set_data(range(len(sim.infected_count)), sim.infected_count)
    line_r.set_data(range(len(sim.recovered_count)), sim.recovered_count)
    
    # Update title with current counts
    s_count = sim.susceptible_count[-1]
    i_count = sim.infected_count[-1]
    r_count = sim.recovered_count[-1]
    
    ax1.set_title(f'Step {frame}: S={s_count}, I={i_count}, R={r_count}')
    
    return scat,

# Create and run animation
ani = FuncAnimation(fig, animate, frames=num_steps, interval=100, blit=False)

# Display the animation
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy._typing import _128Bit
from scipy.spatial.distance import pdist, squareform
import random
import pandas as pd
import itertools
from matplotlib.gridspec import GridSpec
import seaborn as sns

class ParametricFlockingInfectionSimulation:
    def __init__(self, infection_chance, infection_duration, initial_infections, 
                 neighbor_threshold, seed=None,
                 num_steps=150,
                 num_agents=100,
                 L = 10,
                 v = .1,
                 eta = .15,
                 alignment_strength = .7):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        
            
        # Store parameters
        self.infection_chance = infection_chance
        self.infection_duration = infection_duration
        self.initial_infections = initial_infections
        self.neighbor_threshold = neighbor_threshold
        self.num_steps = num_steps
        self.num_agents = num_agents
        self.L = L
        self.v = v
        self.eta = eta
        self.alignment_strength = alignment_strength
        
        # Initialize positions and directions
        self.positions = np.random.rand(num_agents, 2) * L
        self.directions = np.random.rand(num_agents) * 2 * np.pi
        
        # Initialize infection status
        self.health_status = {
            "infected": set(random.sample(list(range(num_agents)), initial_infections)),
            "recovered": set()
        }
        self.infection_status = {}
        for i in self.health_status["infected"]:
            self.infection_status[i] = 0
            
        # Data collection
        self.susceptible_count = []
        self.infected_count = []
        self.recovered_count = []
    
    def get_nearest_neighbors(self, k=6):
        """Get the k nearest neighbors for each agent"""
        distances = squareform(pdist(self.positions))
        np.fill_diagonal(distances, np.inf)
        nearest_indices = np.argsort(distances, axis=1)[:, :k]
        return nearest_indices
    
    def update_flocking(self):
        """Update positions and directions based on flocking rules"""
        new_positions = np.zeros_like(self.positions)
        new_directions = np.zeros_like(self.directions)
        
        nearest_neighbors = self.get_nearest_neighbors(k=6)
        
        for i in range(self.num_agents):
            neighbors = nearest_neighbors[i]
            neighbor_directions = self.directions[neighbors]
            
            neighbor_vectors = np.column_stack([
                np.cos(neighbor_directions),
                np.sin(neighbor_directions)
            ])
            avg_direction_vector = np.mean(neighbor_vectors, axis=0)
            avg_direction = np.arctan2(avg_direction_vector[1], avg_direction_vector[0])
            
            new_directions[i] = ((1 - alignment_strength) * self.directions[i] + 
                               alignment_strength * avg_direction)
            new_directions[i] += eta * (np.random.rand() - 0.5)
            
            new_positions[i] = self.positions[i] + v * np.array([
                np.cos(new_directions[i]), 
                np.sin(new_directions[i])
            ])
            new_positions[i] %= L
        
        self.positions[:] = new_positions
        self.directions[:] = new_directions
    
    def update_infection(self):
        """Update infection status"""
        nearest_neighbors = self.get_nearest_neighbors(k=self.neighbor_threshold)
        new_infections = []
        newly_recovered = []
        
        for i in list(self.health_status["infected"]):
            if self.infection_status[i] >= self.infection_duration:
                newly_recovered.append(i)
                continue
            
            for j in nearest_neighbors[i]:
                if (j not in self.health_status["infected"] and 
                    j not in self.health_status["recovered"]):
                    if random.random() < self.infection_chance:
                        new_infections.append(j)
            
            self.infection_status[i] += 1
        
        for agent in newly_recovered:
            self.health_status["infected"].remove(agent)
            self.health_status["recovered"].add(agent)
            del self.infection_status[agent]
        
        self.health_status["infected"].update(new_infections)
        for agent in new_infections:
            self.infection_status[agent] = 0
    
    def collect_data(self):
        """Collect population counts"""
        susceptible = num_agents - len(self.health_status["infected"]) - len(self.health_status["recovered"])
        infected = len(self.health_status["infected"])
        recovered = len(self.health_status["recovered"])
        
        self.susceptible_count.append(susceptible)
        self.infected_count.append(infected)
        self.recovered_count.append(recovered)
    
    def run_simulation(self):
        """Run the complete simulation and return data"""
        self.collect_data()
        for step in range(num_steps):
            self.update_flocking()
            self.update_infection()
            self.collect_data()
        
        return {
            'susceptible': np.array(self.susceptible_count),
            'infected': np.array(self.infected_count),
            'recovered': np.array(self.recovered_count),
            'time': np.arange(len(self.susceptible_count)),
            'parameters': {
                'infection_chance': self.infection_chance,
                'infection_duration': self.infection_duration,
                'initial_infections': self.initial_infections,
                'neighbor_threshold': self.neighbor_threshold
            }
        }

def run_parameter_sweep():
    """Run comprehensive parameter sweep"""
    
    # Define parameter ranges
    infection_chances = [0.1, 0.2, 0.3, 0.4, 0.5]
    infection_durations = [10, 15, 20, 25]
    initial_infection_counts = [1, 3, 5, 8]
    neighbor_thresholds = [3, 6, 9]
    
    # Number of replications per parameter combination
    num_replications = 5
    
    # Generate all parameter combinations
    param_combinations = list(itertools.product(
        infection_chances, infection_durations, 
        initial_infection_counts, neighbor_thresholds
    ))
    
    total_runs = len(param_combinations) * num_replications
    print(f"Running {total_runs} simulations ({len(param_combinations)} parameter combinations × {num_replications} replications)")
    
    all_results = []
    run_count = 0
    
    for params in param_combinations:
        infection_chance, infection_duration, initial_infections, neighbor_threshold = params
        
        for rep in range(num_replications):
            run_count += 1
            if run_count % 50 == 0:
                print(f"Completed {run_count}/{total_runs} runs ({run_count/total_runs*100:.1f}%)")
            
            # Use unique seed for each run
            seed = run_count
            
            sim = ParametricFlockingInfectionSimulation(
                infection_chance=infection_chance,
                infection_duration=infection_duration,
                initial_infections=initial_infections,
                neighbor_threshold=neighbor_threshold,
                seed=seed
            )
            
            result = sim.run_simulation()
            result['replication'] = rep
            result['run_id'] = run_count
            all_results.append(result)
    
    print("Parameter sweep completed!")
    return all_results

In [ ]:





def analyze_parameter_effects(results):
    """Comprehensive analysis of parameter effects"""
    
    # Extract summary statistics for each run
    summary_data = []
    
    for result in results:
        params = result['parameters']
        
        # Calculate key metrics
        peak_infected = np.max(result['infected'])
        time_to_peak = np.argmax(result['infected'])
        final_recovered = result['recovered'][-1]
        attack_rate = final_recovered / num_agents
        
        # Calculate epidemic duration (time until no more infected)
        infected_counts = result['infected']
        epidemic_end = len(infected_counts) - 1
        for i in range(len(infected_counts)-1, -1, -1):
            if infected_counts[i] > 0:
                epidemic_end = i
                break
        
        summary_data.append({
            'infection_chance': params['infection_chance'],
            'infection_duration': params['infection_duration'],
            'initial_infections': params['initial_infections'],
            'neighbor_threshold': params['neighbor_threshold'],
            'replication': result['replication'],
            'run_id': result['run_id'],
            'peak_infected': peak_infected,
            'time_to_peak': time_to_peak,
            'final_recovered': final_recovered,
            'attack_rate': attack_rate,
            'epidemic_duration': epidemic_end
        })
    
    df = pd.DataFrame(summary_data)
    
    # Create comprehensive visualizations
    fig = plt.figure(figsize=(20, 15))
    gs = GridSpec(4, 4, figure=fig)
    
    # 1. Attack rate vs infection chance
    ax1 = fig.add_subplot(gs[0, 0])
    df_grouped = df.groupby('infection_chance')['attack_rate'].agg(['mean', 'std']).reset_index()
    ax1.errorbar(df_grouped['infection_chance'], df_grouped['mean'], 
                yerr=df_grouped['std'], marker='o', capsize=5)
    ax1.set_xlabel('Infection Chance')
    ax1.set_ylabel('Attack Rate')
    ax1.set_title('Attack Rate vs Infection Chance')
    ax1.grid(True, alpha=0.3)
    
    # 2. Peak infected vs infection duration
    ax2 = fig.add_subplot(gs[0, 1])
    df_grouped = df.groupby('infection_duration')['peak_infected'].agg(['mean', 'std']).reset_index()
    ax2.errorbar(df_grouped['infection_duration'], df_grouped['mean'], 
                yerr=df_grouped['std'], marker='s', capsize=5, color='red')
    ax2.set_xlabel('Infection Duration')
    ax2.set_ylabel('Peak Infected')
    ax2.set_title('Peak Infections vs Duration')
    ax2.grid(True, alpha=0.3)
    
    # 3. Time to peak vs initial infections
    ax3 = fig.add_subplot(gs[0, 2])
    df_grouped = df.groupby('initial_infections')['time_to_peak'].agg(['mean', 'std']).reset_index()
    ax3.errorbar(df_grouped['initial_infections'], df_grouped['mean'], 
                yerr=df_grouped['std'], marker='^', capsize=5, color='green')
    ax3.set_xlabel('Initial Infections')
    ax3.set_ylabel('Time to Peak')
    ax3.set_title('Time to Peak vs Initial Infections')
    ax3.grid(True, alpha=0.3)
    
    # 4. Epidemic duration vs neighbor threshold
    ax4 = fig.add_subplot(gs[0, 3])
    df_grouped = df.groupby('neighbor_threshold')['epidemic_duration'].agg(['mean', 'std']).reset_index()
    ax4.errorbar(df_grouped['neighbor_threshold'], df_grouped['mean'], 
                yerr=df_grouped['std'], marker='d', capsize=5, color='purple')
    ax4.set_xlabel('Neighbor Threshold')
    ax4.set_ylabel('Epidemic Duration')
    ax4.set_title('Duration vs Neighbor Threshold')
    ax4.grid(True, alpha=0.3)
    
    # 5-8. Heatmaps showing interactions
    # Attack rate heatmap (infection_chance vs infection_duration)
    ax5 = fig.add_subplot(gs[1, 0])
    pivot_data = df.groupby(['infection_chance', 'infection_duration'])['attack_rate'].mean().unstack()
    sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='Reds', ax=ax5)
    ax5.set_title('Attack Rate Heatmap')
    
    # Peak infected heatmap (initial_infections vs neighbor_threshold)
    ax6 = fig.add_subplot(gs[1, 1])
    pivot_data = df.groupby(['initial_infections', 'neighbor_threshold'])['peak_infected'].mean().unstack()
    sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='Oranges', ax=ax6)
    ax6.set_title('Peak Infected Heatmap')
    
    # Time to peak heatmap
    ax7 = fig.add_subplot(gs[1, 2])
    pivot_data = df.groupby(['infection_chance', 'initial_infections'])['time_to_peak'].mean().unstack()
    sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='Greens', ax=ax7)
    ax7.set_title('Time to Peak Heatmap')
    
    # Duration heatmap
    ax8 = fig.add_subplot(gs[1, 3])
    pivot_data = df.groupby(['infection_duration', 'neighbor_threshold'])['epidemic_duration'].mean().unstack()
    sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='Purples', ax=ax8)
    ax8.set_title('Epidemic Duration Heatmap')
    
    # 9-12. Distribution plots
    ax9 = fig.add_subplot(gs[2, 0])
    ax9.hist(df['attack_rate'], bins=20, alpha=0.7, edgecolor='black')
    ax9.set_xlabel('Attack Rate')
    ax9.set_ylabel('Frequency')
    ax9.set_title('Attack Rate Distribution')
    ax9.axvline(df['attack_rate'].mean(), color='red', linestyle='--', label=f'Mean: {df["attack_rate"].mean():.2f}')
    ax9.legend()
    
    ax10 = fig.add_subplot(gs[2, 1])
    ax10.hist(df['peak_infected'], bins=20, alpha=0.7, color='orange', edgecolor='black')
    ax10.set_xlabel('Peak Infected')
    ax10.set_ylabel('Frequency')
    ax10.set_title('Peak Infected Distribution')
    ax10.axvline(df['peak_infected'].mean(), color='red', linestyle='--', label=f'Mean: {df["peak_infected"].mean():.1f}')
    ax10.legend()
    
    ax11 = fig.add_subplot(gs[2, 2])
    ax11.hist(df['time_to_peak'], bins=20, alpha=0.7, color='green', edgecolor='black')
    ax11.set_xlabel('Time to Peak')
    ax11.set_ylabel('Frequency')
    ax11.set_title('Time to Peak Distribution')
    ax11.axvline(df['time_to_peak'].mean(), color='red', linestyle='--', label=f'Mean: {df["time_to_peak"].mean():.1f}')
    ax11.legend()
    
    ax12 = fig.add_subplot(gs[2, 3])
    ax12.hist(df['epidemic_duration'], bins=20, alpha=0.7, color='purple', edgecolor='black')
    ax12.set_xlabel('Epidemic Duration')
    ax12.set_ylabel('Frequency')
    ax12.set_title('Duration Distribution')
    ax12.axvline(df['epidemic_duration'].mean(), color='red', linestyle='--', label=f'Mean: {df["epidemic_duration"].mean():.1f}')
    ax12.legend()
    
    # 13-16. Example time series for extreme parameter combinations
    # Find most and least severe parameter combinations
    df_param_means = df.groupby(['infection_chance', 'infection_duration', 
                                'initial_infections', 'neighbor_threshold'])['attack_rate'].mean().reset_index()
    most_severe_params = df_param_means.loc[df_param_means['attack_rate'].idxmax()]
    least_severe_params = df_param_means.loc[df_param_means['attack_rate'].idxmin()]
    
    # Plot example time series
    ax13 = fig.add_subplot(gs[3, :2])
    
    # Find results matching most severe parameters
    severe_results = [r for r in results if (
        r['parameters']['infection_chance'] == most_severe_params['infection_chance'] and
        r['parameters']['infection_duration'] == most_severe_params['infection_duration'] and
        r['parameters']['initial_infections'] == most_severe_params['initial_infections'] and
        r['parameters']['neighbor_threshold'] == most_severe_params['neighbor_threshold']
    )]
    
    for i, result in enumerate(severe_results[:3]):  # Show first 3 replications
        ax13.plot(result['time'], result['susceptible'], 'b-', alpha=0.6, linewidth=1)
        ax13.plot(result['time'], result['infected'], 'r-', alpha=0.6, linewidth=1)
        ax13.plot(result['time'], result['recovered'], 'g-', alpha=0.6, linewidth=1)
    
    ax13.set_xlabel('Time Steps')
    ax13.set_ylabel('Number of Agents')
    ax13.set_title(f'Most Severe: p={most_severe_params["infection_chance"]}, d={most_severe_params["infection_duration"]}, i0={most_severe_params["initial_infections"]}, k={most_severe_params["neighbor_threshold"]}')
    ax13.legend(['Susceptible', 'Infected', 'Recovered'])
    ax13.grid(True, alpha=0.3)
    
    ax14 = fig.add_subplot(gs[3, 2:])
    
    # Find results matching least severe parameters
    mild_results = [r for r in results if (
        r['parameters']['infection_chance'] == least_severe_params['infection_chance'] and
        r['parameters']['infection_duration'] == least_severe_params['infection_duration'] and
        r['parameters']['initial_infections'] == least_severe_params['initial_infections'] and
        r['parameters']['neighbor_threshold'] == least_severe_params['neighbor_threshold']
    )]
    
    for i, result in enumerate(mild_results[:3]):  # Show first 3 replications
        ax14.plot(result['time'], result['susceptible'], 'b-', alpha=0.6, linewidth=1)
        ax14.plot(result['time'], result['infected'], 'r-', alpha=0.6, linewidth=1)
        ax14.plot(result['time'], result['recovered'], 'g-', alpha=0.6, linewidth=1)
    
    ax14.set_xlabel('Time Steps')
    ax14.set_ylabel('Number of Agents')
    ax14.set_title(f'Least Severe: p={least_severe_params["infection_chance"]}, d={least_severe_params["infection_duration"]}, i0={least_severe_params["initial_infections"]}, k={least_severe_params["neighbor_threshold"]}')
    ax14.legend(['Susceptible', 'Infected', 'Recovered'])
    ax14.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df

# Run the comprehensive parameter sweep
print("Starting comprehensive parameter sweep...")
results = run_parameter_sweep()

# Analyze results
print("\nAnalyzing parameter effects...")
summary_df = analyze_parameter_effects(results)

# Save detailed results
detailed_data = []
for result in results:
    params = result['parameters']
    for t, (s, i, r) in enumerate(zip(result['susceptible'], result['infected'], result['recovered'])):
        detailed_data.append({
            'run_id': result['run_id'],
            'replication': result['replication'],
            'infection_chance': params['infection_chance'],
            'infection_duration': params['infection_duration'],
            'initial_infections': params['initial_infections'],
            'neighbor_threshold': params['neighbor_threshold'],
            'time_step': t,
            'susceptible': s,
            'infected': i,
            'recovered': r
        })

detailed_df = pd.DataFrame(detailed_data)
summary_df.to_csv('parameter_sweep_summary.csv', index=False)
detailed_df.to_csv('parameter_sweep_detailed.csv', index=False)

print(f"\nParameter sweep completed!")
print(f"Summary data: {summary_df.shape}")
print(f"Detailed data: {detailed_df.shape}")
print("Data saved to 'parameter_sweep_summary.csv' and 'parameter_sweep_detailed.csv'")

# Print key findings
print("\n" + "="*80)
print("KEY FINDINGS FROM PARAMETER SWEEP")
print("="*80)

print(f"\nOverall Statistics:")
print(f"  Total simulations run: {len(summary_df)}")
print(f"  Parameter combinations tested: {len(summary_df.groupby(['infection_chance', 'infection_duration', 'initial_infections', 'neighbor_threshold']))}")
print(f"  Mean attack rate across all runs: {summary_df['attack_rate'].mean():.3f} ± {summary_df['attack_rate'].std():.3f}")
print(f"  Mean peak infections: {summary_df['peak_infected'].mean():.1f} ± {summary_df['peak_infected'].std():.1f}")

print(f"\nParameter Effects (correlation with attack rate):")
correlations = summary_df[['infection_chance', 'infection_duration', 'initial_infections', 
                          'neighbor_threshold', 'attack_rate']].corr()['attack_rate'].sort_values(ascending=False)
for param, corr in correlations.items():
    if param != 'attack_rate':
        print(f"  {param}: {corr:.3f}")

print(f"\nExtreme Cases:")
max_attack = summary_df.loc[summary_df['attack_rate'].idxmax()]
min_attack = summary_df.loc[summary_df['attack_rate'].idxmin()]
print(f"  Highest attack rate: {max_attack['attack_rate']:.3f}")
print(f"    Parameters: p={max_attack['infection_chance']}, d={max_attack['infection_duration']}, i0={max_attack['initial_infections']}, k={max_attack['neighbor_threshold']}")
print(f"  Lowest attack rate: {min_attack['attack_rate']:.3f}")
print(f"    Parameters: p={min_attack['infection_chance']}, d={min_attack['infection_duration']}, i0={min_attack['initial_infections']}, k={min_attack['neighbor_threshold']}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sir_model(y, t, beta, gamma):
    """SIR ODE model"""
    S, I, R = y
    N = S + I + R
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]

def compare_simulation_with_ode(beta=0.3, gamma=0.05, seed=42, infection_chance=0.3, infection_duration=10, initial_infections=5, neighbor_infection_threshold=6,
num_steps=150,
num_agents=100,
L = 10,
v = .1,
eta = .15,
alignment_strength = .7):
    """
    Run FlockingInfectionSimulation and compare with ODE model
    """
    
    # Run the agent-based simulation
    print("Running agent-based simulation...")
    sim = ParametricFlockingInfectionSimulation(infection_chance=infection_chance, infection_duration=infection_duration, initial_infections=initial_infections, neighbor_threshold=neighbor_infection_threshold,
    num_steps=num_steps,
    num_agents=num_agents,
    L = L,
    v = v,
    eta = eta,
    alignment_strength = alignment_strength)
    if hasattr(sim, '__init__') and 'seed' in sim.__init__.__code__.co_varnames:
        sim = ParametricFlockingInfectionSimulation(infection_chance=infection_chance, infection_duration=infection_duration, initial_infections=initial_infections, seed=seed, neighbor_threshold=neighbor_infection_threshold,
        num_steps=num_steps,
        num_agents=num_agents,
        L = L,
        v = v,
        eta = eta,
        alignment_strength = alignment_strength)
    
    # Get simulation results
    sim_results = sim.run_simulation()
    
    # Extract simulation data
    time_sim = sim_results['time']
    S_sim = sim_results['susceptible']
    I_sim = sim_results['infected'] 
    R_sim = sim_results['recovered']
    
    print(f"Simulation completed: {len(time_sim)} time steps")
    
    # Run ODE model with same initial conditions
    print("Running ODE model...")
    y0 = [S_sim[0], I_sim[0], R_sim[0]]  # Same initial conditions as simulation
    t_ode = np.linspace(0, len(time_sim)-1, len(time_sim))  # Same time points
    
    sol = odeint(sir_model, y0, t_ode, args=(beta, gamma))
    S_ode, I_ode, R_ode = sol.T
    
    # Create side-by-side comparison plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Agent-based simulation
    ax1.plot(time_sim, S_sim, 'b-', linewidth=2, label='Susceptible', alpha=0.8)
    ax1.plot(time_sim, I_sim, 'r-', linewidth=2, label='Infected', alpha=0.8)
    ax1.plot(time_sim, R_sim, 'g-', linewidth=2, label='Recovered', alpha=0.8)

    
    ax1.set_xlabel('Time Steps')
    ax1.set_ylabel('Number of Agents')
    ax1.set_title('Agent-Based Flocking + Infection Simulation')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, max(S_sim[0], I_sim[0], R_sim[0]) + 5)

    ax1.set_xlim(0, 80)
    
    # Add simulation statistics
    peak_I_sim = np.max(I_sim)
    time_peak_sim = time_sim[np.argmax(I_sim)]
    final_R_sim = R_sim[-1]
    
    ax1.text(0.02, 0.98, f'Peak I: {peak_I_sim:.0f}\nTime to peak: {time_peak_sim:.0f}\nFinal R: {final_R_sim:.0f}',
             transform=ax1.transAxes, verticalalignment='top',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
    
    # Plot 2: ODE model
    ax2.plot(t_ode, S_ode, 'b-', linewidth=2, label='Susceptible', alpha=0.8)
    ax2.plot(t_ode, I_ode, 'r-', linewidth=2, label='Infected', alpha=0.8)
    ax2.plot(t_ode, R_ode, 'g-', linewidth=2, label='Recovered', alpha=0.8)
    
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Population')
    ax2.set_title(f'SIR ODE Model (β={beta}, γ={gamma}, R₀={beta/gamma:.2f})')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, max(S_sim[0], I_sim[0], R_sim[0]) + 5)
    
    # Add ODE statistics
    peak_I_ode = np.max(I_ode)
    time_peak_ode = t_ode[np.argmax(I_ode)]
    final_R_ode = R_ode[-1]
    
    ax2.text(0.02, 0.98, f'Peak I: {peak_I_ode:.0f}\nTime to peak: {time_peak_ode:.1f}\nFinal R: {final_R_ode:.0f}',
             transform=ax2.transAxes, verticalalignment='top',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print comparison
    print("\n" + "="*60)
    print("SIMULATION vs ODE COMPARISON")
    print("="*60)
    print(f"Peak Infections:")
    print(f"  Simulation: {peak_I_sim:.0f} at time {time_peak_sim:.0f}")
    print(f"  ODE:        {peak_I_ode:.0f} at time {time_peak_ode:.1f}")
    print(f"  Difference: {abs(peak_I_sim - peak_I_ode):.0f}")
    
    print(f"\nFinal Recovered:")
    print(f"  Simulation: {final_R_sim:.0f}")
    print(f"  ODE:        {final_R_ode:.0f}")
    print(f"  Difference: {abs(final_R_sim - final_R_ode):.0f}")
    
    # Calculate overall fit quality
    mse_I = np.mean((I_sim - I_ode)**2)
    mse_total = (np.mean((S_sim - S_ode)**2) + np.mean((I_sim - I_ode)**2) + np.mean((R_sim - R_ode)**2))/3
    
    print(f"\nFit Quality:")
    print(f"  MSE (Infected): {mse_I:.2f}")
    print(f"  MSE (Overall):  {mse_total:.2f}")
    
    return sim_results, (t_ode, S_ode, I_ode, R_ode)


sim_data, ode_data = compare_simulation_with_ode(infection_chance=0.15, infection_duration=10, initial_infections=6, neighbor_infection_threshold=5, num_steps=10)
